# [LVV-2668] Measure torque capacity of the CCW drive

This notebook is used to measure the torque capacity of the CCW.

Requirements
* EFD avaliable
* Rotator powered on
* Thermal sensors attached to the two rotator motors
* CSC running
* CCW in following mode

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures by using the chronograph during several steps is required.

The first parts of this notebook are copied from LVV-2344 'Startup MT Components for System Spread Integration Tests on Level 3'

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

[LVV-2668]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2668
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

## Setting Up Test Environment

Before we run the tests, we want to make sure that we have all the libraries imported, remotes connected, etc.

In [ ]:
test_message = "CCW Torque Capacity Test"
test_case = "LVV-T2668"
test_exec = "LVV-EXXXX"

# Put date in DDDD
script_id = DDDD2688

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging

import pandas as pd

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS

from lsst.sitcom import vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

### Check environment setup

The following cell will print some of the basic DDS configutions.

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID", "Expected, not set."))

### Setup logging

Setup logging in debug mode and create a logger to use on the notebook.

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

### Starting communication resources

We start by creating a domain and later instantiate the MTCS class.
We will use the class to startup the components. 

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

# Starting components

From now on we will start the various components of the MTAOS.
You may wonder why are we not simply sending all CSCs to ENABLED state in one go, as we usually do on other systems.

The answer is that the MTCS components have some initilization dependencies that need to be observed for the components to be enabled properly.
We will describe these as we work our way the initialization steps.


## Starting MTPtg

We start by making sure the pointing component is alive, by waiting for a heartbeat.
Next we enable the component using `mtcs.set_state` method.

We select to start with the `MTPtg` mainly because, of all components of the `MTCS` it is the only pure-software components.
As such the `MTPtg` is pretty independent and can be brought to enabled in any condition.

It is also worth noticed that, as a pure-software component, the `MTPtg` does not have a simulation mode.

Furthermore, as you will notice below, we are not checking the software version of the `MTPtg`, mainly because the component is currently not sending this information.

In [ ]:
await mtcs.next_heartbeat("mtptg")

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtptg"])

## Starting MTMount

This is one case where the initialization order is important. 

The MTMount needs to be enabled before we enable the MTRotator.
The reason is that the MTRotator needs to know the position of the Camera Cable Wrap (CCW), which is provided by the MTMount, before it can be enable. 
If the MTRotator does not receive the position of the CCW, it will immediatelly activate the breaks and transition to FAULT state.

We start by verifying that the CSC is sending heartbeats.

In [ ]:
await mtcs.next_heartbeat("mtmount")

Now we can enable the CSC.

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtmount"])

### Perform some basic checks

The following are a couple of sanity checks we routinely perform when starting the MTMount.

We check if the CSC is running in simulation mode and then the version of the CSC.

Finally, we verify that the camera cable wrap following is enabled.

In [ ]:
mtmount_simulation_mode = await mtcs.get_simulation_mode(["mtmount"])

mode = mtmount_simulation_mode["mtmount"].mode
timestamp = pd.to_datetime(mtmount_simulation_mode["mtmount"].private_sndStamp, unit='s')

log.debug(
    f"MTMount simulation mode: {mode} @ {timestamp}"
)

In [ ]:
mtmount_software_versions = await mtcs.get_software_versions(["mtmount"])

csc_version = mtmount_software_versions["mtmount"].cscVersion
timestamp = pd.to_datetime(mtmount_software_versions["mtmount"].private_sndStamp, unit='s')

log.debug(
    f"MTMount software version: {csc_version} @ {timestamp}",
)

In [ ]:
mtmount_ccw_following = await mtcs.rem.mtmount.evt_cameraCableWrapFollowing.aget()

timestamp = pd.to_datetime(mtmount_ccw_following.private_sndStamp, unit='s')

if mtmount_ccw_following.enabled:
    log.debug(f"CCW following mode enabled: {mtmount_ccw_following.enabled} @ {timestamp}.")
else:
    await mtcs.set_state(salobj.State.DISABLED, ["mtmount"])
    raise RuntimeError(
        "CCW following mode not enabled. Usually this means that the MTMount could "
        "not see telemetry from the rotator when it was enabled. To correct this condition "
        "make sure the MTRotator telemetry is being published, then execute the procedure again. "
        "MTMount CSC will be left in DISABLED state."
        )


## Starting Rotator

In [ ]:
await mtcs.next_heartbeat("mtrotator")

In [ ]:
await mtcs.set_state(salobj.State.ENABLED, components=["mtrotator"])

### Perform some basic checks

The following is a few sanity checks we routinely perform to verify the system integrity at this stage.

In [ ]:
mtrotator_simulation_mode = await mtcs.get_simulation_mode(["mtrotator"])

mode = mtrotator_simulation_mode["mtrotator"].mode
timestamp = pd.to_datetime(mtrotator_simulation_mode["mtrotator"].private_sndStamp, unit='s')

log.debug(
    f"MTRotator simulation mode: {mode} @ {timestamp}"
)

In [ ]:
mtrotator_software_versions = await mtcs.get_software_versions(["mtrotator"])

csc_version = mtrotator_software_versions["mtrotator"].cscVersion
timestamp = pd.to_datetime(mtrotator_software_versions["mtrotator"].private_sndStamp, unit='s')

log.debug(
    f"MTRotator software version: {csc_version} @ {timestamp}",
)

In [ ]:
elevation = await mtcs.rem.mtmount.tel_elevation.next(flush=True, timeout=5)
azimuth = await mtcs.rem.mtmount.tel_azimuth.next(flush=True, timeout=5)
ccw = await mtcs.rem.mtmount.tel_cameraCableWrap.next(flush=True, timeout=5)
rotator = await mtcs.rem.mtrotator.tel_rotation.next(flush=True, timeout=5)

log.info(f"mount elevation Angle = {elevation.actualPosition}")
log.info(f"mount azimuth angle = {azimuth.actualPosition}")
log.info(f"CCW angle = {ccw.actualPosition}. Needs to be within 2.2 deg of rotator angle ")
log.info(f"rot angle = {rotator.actualPosition} diff = {rotator.actualPosition - ccw.actualPosition}")

### CCW telemetry too old

This warning message may appear in the `MTRotator` in a couple different conditions.

The most common occurence is when the `MTMount` component is not publishing the CCW telemetry.
This should be rectified by enabling the CSC, as we've done on the section above, and is one of the reasons we enable `MTMount` before the `MTRotator`.

The less common but more critical condition is when the clock on the `MTMount` controller is out of sync with the observatory clock server.
In this case, the `timestamp` attribute, used by the `MTRotator` to determine the relevant time for the published telemetry, will be out of sync and we won't be able to operate the system.

You can use the cell below to determine whether this is the case or not.
If so, you need to contact IT or someone with knowledge about the `MTMount` low level controller to fix the time synchronization issue.



In [ ]:
ccw = await mtcs.rem.mtmount.tel_cameraCableWrap.next(flush=True, timeout=5)
rotator = await mtcs.rem.mtrotator.tel_rotation.next(flush=True, timeout=5)

ccw_snd_stamp = pd.to_datetime(ccw.private_sndStamp, unit='s')
ccw_timestamp = pd.to_datetime(ccw.timestamp, unit='s')
ccw_actual_position = ccw.actualPosition

rotator_snd_stamp = pd.to_datetime(rotator.private_sndStamp, unit='s')
rotator_timestamp = pd.to_datetime(rotator.timestamp, unit='s')
rotator_actual_position = rotator.actualPosition

log.info(
    f"CCW:: snd_stamp={ccw_snd_stamp} timestamp={ccw_timestamp} actual position={ccw_actual_position}"
    )
log.info(
    f"Rotator:: snd_stamp={rotator_snd_stamp} timestamp={rotator_timestamp} actual position={rotator_actual_position}"
    )

ccw_telemetry_maximum_age = pd.to_timedelta(1.0, unit='s')

if abs(ccw_snd_stamp - ccw_timestamp) > ccw_telemetry_maximum_age:
    log.warning(
        f"CCW timestamp out of sync by {abs(ccw_snd_stamp - ccw_timestamp)}s. "
        "System may not work. Check clock synchronization in MTMount low level controller."
        )

### Clearing error in MTRotator

If the MTRotator is in FAULT state, you need to send the `clearError` command before transitioning it back to `ENABLED`.

This is a particularity of the `MTRotator` (and `MTHexapod`) that violates our state machine.

In [ ]:
if False:
    await mtcs.rem.mtrotator.cmd_clearError.set_start()

## Starting Camera Hexapod

In [ ]:
await mtcs.next_heartbeat("mthexapod_1")

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mthexapod_1"]
    )

In [ ]:
mthexapod_1_simulation_mode = await mtcs.get_simulation_mode(["mthexapod_1"])

mode = mthexapod_1_simulation_mode["mthexapod_1"].mode
timestamp = pd.to_datetime(mthexapod_1_simulation_mode["mthexapod_1"].private_sndStamp, unit='s')

log.debug(
    f"Camera Hexapod simulation mode: {mode} @ {timestamp}"
)

In [ ]:
mthexapod_1_software_versions = await mtcs.get_software_versions(["mthexapod_1"])

csc_version = mthexapod_1_software_versions["mthexapod_1"].cscVersion
timestamp = pd.to_datetime(mthexapod_1_software_versions["mthexapod_1"].private_sndStamp, unit='s')

log.debug(
    f"Camera Hexapod software version: {csc_version} @ {timestamp}",
)

In [ ]:
if False:
    await mtcs.rem.mthexapod_1.cmd_clearError.set_start()

In [ ]:
await mtcs.enable_compensation_mode(component="mthexapod_1")

In [ ]:
await mtcs.reset_camera_hexapod_position()

### Make EFD connection for later

In [ ]:
client = vandv.efd.create_efd_client()

### Start a script controller for custom log messages to use for later EFD analyses.

In [ ]:
start_time = datetime.now()

script = salobj.Controller("Script", index=scriptID)
await asyncio.sleep(10) # May help with DDS problems; closing all other kernels may help too
print(f"{test_case} {test_exec} time to start is {datetime.now() - start_time} [s]")

## Begin Tests

First: verify the MTRotator_logevent_commandableByDDS event is True.

In [ ]:
await asyncio.sleep(2)

data = rotator.evt_commandableByDDS.get()
print(data.state)

Make sure we start in position zero.

In [ ]:
script.log.info(f"{test_message} -- {test_case} {test_exec} TESTING BEGINS")

script.log.info(f"START - {test_message} -- {test_case} {test_exec} Reset Position to zero")
await rotator.cmd_move.set_start(position=0, timeout=90)
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} Reset Position to zero")

#### Move from 0 to +88 degrees

In [ ]:
script.log.info(f"START - {test_message} -- {test_case} {test_exec} 0 to 88 deg")
await rotator.cmd_move.set_start(position=88, timeout=90)
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} 0 to 88 deg")

Cool for close to two minutes and confirm that the rotator temperature is less than 25 degrees by manually checking the chronograph. Note these entries will not exist on the Tucson Test Stand.

Look at the lsst.sal.ESS.temperature temperature6 and temperature7 entries.  They correspond to the temperature of the two rotator motors.
Enter the values here:

|Motor 1 (C)|Motor 2 (C) |
|--------------|---------------|
| 0 | 0 |

Now confirm that the actualTorquePercentage0 and actualTorquePercentage1 variables from the lsst.sal.MTMount.cameraCableWrap are being published to the EFD.  We will need this for later analysis.

In [ ]:
df = await client.select_top_n("lsst.sal.MTMount.cameraCableWrap", 
                               fields=['actualTorquePercentage0', 'actualTorquePercentage1'], 
                               num=10)
print(df)

#### Move from +88 to +0 degrees

In [ ]:
script.log.info(f"START - {test_message} -- {test_case} {test_exec} 88 to 0 deg")
await rotator.cmd_move.set_start(position=0, timeout=90)
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} 88 to 0 deg")

Cool for close to two minutes and confirm that the rotator temperature is less than 25 degrees by manually checking the chronograph and entring the values here:


|Motor 1 (C)|Motor 2 (C) |
|--------------|---------------|
| 0 | 0 |

#### Move from 0 to -88 degrees

In [ ]:
script.log.info(f"START - {test_message} -- {test_case} {test_exec} 0 to -88 deg")
await rotator.cmd_move.set_start(position=-88, timeout=90)
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} 0 to -88 deg")

Cool for close to two minutes and confirm that the rotator temperature is less than 25 degrees by manually checking the chronograph and entring the values here:


|Motor 1 (C)|Motor 2 (C) |
|--------------|---------------|
| 0 | 0 |

#### Move from -88 to +88 degrees

In [ ]:
script.log.info(f"START - {test_message} -- {test_case} {test_exec} -88 to 88 deg")
await rotator.cmd_move.set_start(position=88, timeout=90)
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} -88 to 88 deg")

Cool for close to two minutes and confirm that the rotator temperature is less than 25 degrees by manually checking the chronograph and entring the values here:


|Motor 1 (C)|Motor 2 (C) |
|--------------|---------------|
| 0 | 0 |

#### Move from +88 to 0 degrees

In [ ]:
script.log.info(f"START - {test_message} -- {test_case} {test_exec} -88 to 0 deg")
await rotator.cmd_move.set_start(position=0, timeout=90)
script.log.info(f"STOP - {test_message} -- {test_case} {test_exec} -88 to 0 deg")

Cool for close to two minutes and confirm that the rotator temperature is less than 25 degrees by manually checking the chronograph and entring the values here:


|Motor 1 (C)|Motor 2 (C) |
|--------------|---------------|
| 0 | 0 |

### End tests

In [ ]:
script.log.info(f"{test_message} -- {test_case} {test_exec} TESTING ENDS")
stop_time = datetime.now()

# Make simple analysis plots to confirm the test worked.

In [ ]:
df_torques = await client.select_time_series('lsst.sal.MTMount.cameraCableWrap', 
                                             fields=['actualTorquePercentage0', 'actualTorquePercentage1'], 
                                             start=time_start, end=time_end))

df_rotator = await client.select_time_series('lsst.sal.MTRotator.rotation',
                                             fields='actualPosition', 
                                             start=time_start, end=time_end)

In [ ]:
# Plot to position of the rotator during this time.
df_rotator.plot()

In [ ]:
# This should plot both torque percentages 
df_torques.plot()

---
# Closing MTCS and Domain

You can use the commands below to easily shut-down (send to STANDBY) all the components.

In [ ]:
await mtcs.standby()

In [ ]:
await mtcs.close()

In [ ]:
await domain.close()